# Pandas Cheatsheet
---
## What is `Pandas`?
Library built on top of `numpy` that provides a framework for data manipulation and analysis. It is named after **Panel Data** which is a term for multidimensional structured data. 

This library contains functions and utilities needed to handle large, imbalanced, and inconsisten datasets. It comes in handy in the Data Acquisition, Exploration and Preprocessing stages of Machine Learning.

It makes heavy used of `Series` and `DataFrame` data structure to store multidimensional data.
* `Series` 
    * built on top of `numpy` arrays
    * 1-dimensional 
    * each data-point has an associated `label` 
    * collection of these labels is called an `index`
* `DataFrame`
    * 2-dimensional
    * similiar to a spreadsheet in excel
    * essentially a Python dictionary that maps column names (keys) to their associated `Series` (values)
* `Panel`
    * 3-dimensional
* `Panel4D`
    * 4-dimensional

## Importing `pandas`

In [227]:
import pandas as pd

# also import numpy for examples
import numpy as np

## Series and DataFrame Basics

### Series
* Series can hold multiple data types 
* They can be indexed by numeric and non-numeric labels
* They can be created using Python lists, numpy arrays, and dictionaries.

In [228]:
index_labels = ['first', 'second', 'third']
list = [1, 10, 100]
np_array = np.array([2,20,200])
dict = {'first': 3, 'second': 30, 'third': 300}

# create series with python list and default 0-based indices
print(pd.Series(data=list)) 

# create series with numpy array and custom indices
print(pd.Series(data=np_array, index=index_labels)) 

# create series with python dictionary with custom indices (dictionary keys becomes the indices)
print(pd.Series(data=dict))


0      1
1     10
2    100
dtype: int64
first       2
second     20
third     200
dtype: int64
first       3
second     30
third     300
dtype: int64


Arithmetic can also be performed on Series instances

In [229]:
s1 = pd.Series(data=[10, 20, 30])
s2 = pd.Series(data=[40, 50, 60])
print(s1 + s2)     # Peform element-wise addition based on matching indices

# NOTE: Element-wise summations occur for elements with matching indices across the 2 series.
# If one operand is missing (e.g., one series doesn't have an associated value for the index), NaN
# is used. Specify a fill_value to prevent this
s3 = pd.Series(data=[10, 20, 30], index=[1,2,3])
s4 = pd.Series(data=[20, 30, 40], index=[2,3,4])
print(s3 + s4)
print(s3.add(s4, fill_value=0))

0    50
1    70
2    90
dtype: int64
1     NaN
2    40.0
3    60.0
4     NaN
dtype: float64
1    10.0
2    40.0
3    60.0
4    40.0
dtype: float64


### Data Frames

## Retrieving/Reading Data

See https://pandas.pydata.org/docs/reference/io.html for additional data I/O functions such as reading/writing to and from excel, json, and more.

In [230]:
# Read CSV
data = pd.read_csv('./example.csv')
data

,name,age,salary_usd,profession,employer
0,spongebob,19,12000.0,chef,the krusty krab
1,plankton,45,2.0,entrepreneur,the chum bucket
2,sandy,24,65000.0,retired,none
3,mr. krabs,41,100000.0,entrepreneur,the krusty krab
4,larry,25,37000.0,bodybuilder,none
5,patrick star,31,0.0,unemployed,none


## Data

### Inspecting 

Use the following to take a peek at the dataset
* `DataFrame.head()`
* `DataFrame.tail()`


In [231]:
data.head(2)

,name,age,salary_usd,profession,employer
0,spongebob,19,12000.0,chef,the krusty krab
1,plankton,45,2.0,entrepreneur,the chum bucket


In [232]:
data.tail(2)

,name,age,salary_usd,profession,employer
4,larry,25,37000.0,bodybuilder,none
5,patrick star,31,0.0,unemployed,none


In [233]:
data.describe()     # get high level metrics for numeric columns

,age,salary_usd
count,6.000000,6.000000
mean,30.833333,35667.000000
std,10.245324,40262.117853
min,19.000000,0.000000
25%,24.250000,3001.500000
50%,28.000000,24500.000000
75%,38.500000,58000.000000
max,45.000000,100000.000000


In [234]:
data.shape

(6, 5)

In [235]:
data.dtypes

name           object
age             int64
salary_usd    float64
profession     object
employer       object
dtype: object

### Converting Column Types

We need to convert the columns to the correct types to help future processing tasks and data integrity


In [236]:
# Convert columns one by one
data['age'] = pd.to_numeric(data['age'], downcast="integer", errors="coerce")
data['salary_usd'] = pd.to_numeric(data['salary_usd'], downcast='float', errors='coerce')
print(data.dtypes)

name           object
age              int8
salary_usd    float32
profession     object
employer       object
dtype: object


We could have also done this at `read_csv()` time by passing a types dictionary

In [237]:
data = pd.read_csv('./example.csv', dtype={
    'name': str,
    'age': int,
    'salary_usd': float,
    'profession': str,
    'employer': str
})
data.dtypes

name           object
age             int64
salary_usd    float64
profession     object
employer       object
dtype: object

### Filtering/Indexing

#### Indexes

All `Series`, `DataFrame`, `Panel` and `Panel4D` are indexed via indexes. These can be numeric or non-numeric (ex. `category`, `type`) indexes. They are extremely helpful in allowing for rapid indexing into multi-dimensional data and clean organization. 

They also help with emulating higher-dimensional data within 2 dimensional data (i.e., `DataFrame`)

Helpful Resources:
* https://towardsdatascience.com/understand-pandas-indexes-1b94f5c078c6
* https://stackoverflow.com/questions/13226029/benefits-of-pandas-multiindex#answer-13226352

#### Rows

You can filter rows based on labels (numeric and non-numeric) labels:
* Use `DataFrame.loc[]` for fetching columns based on the string label
* Use `DataFrame.iloc[]` for fetching columns based on the numeric label

In [238]:
# Fetch by numeric label
data.iloc[1:3]        # our labels are int indexes (See code above) there we use `iloc[]`...

,name,age,salary_usd,profession,employer
1,plankton,45,2.0,entrepreneur,the chum bucket
2,sandy,24,65000.0,retired,none


In [239]:
data.loc[3]

name                mr. krabs
age                        41
salary_usd             100000
profession       entrepreneur
employer      the krusty krab
Name: 3, dtype: object

#### Columns

In [240]:
name_and_profession = data[['name', 'profession']]
name_and_profession

,name,profession
0,spongebob,chef
1,plankton,entrepreneur
2,sandy,retired
3,mr. krabs,entrepreneur
4,larry,bodybuilder
5,patrick star,unemployed


We can also refer to single columns as a property of the DataFrame:

In [241]:
data.name

0       spongebob
1        plankton
2           sandy
3       mr. krabs
4           larry
5    patrick star
Name: name, dtype: object

#### Boolean Indexing

Boolean Indexing relies indexing into a Series or DataFrame based on a sequence of boolean values.

If we generate a Series of boolean values and feed this into a DataFrame indexer, then Pandas will
retrieve all the rows associated with the True boolean indexes

In [242]:
# Long-hand version
boolean_index = data['age'] > 40    # this element-wise operation will generate a series of boolean values

old_people = data[boolean_index]    # use the boolean index within dataframe indexer to retrieve the rows
old_people


,name,age,salary_usd,profession,employer
1,plankton,45,2.0,entrepreneur,the chum bucket
3,mr. krabs,41,100000.0,entrepreneur,the krusty krab


In [243]:
# short-hand version
young_people = data[data.age < 40]
young_people

,name,age,salary_usd,profession,employer
0,spongebob,19,12000.0,chef,the krusty krab
2,sandy,24,65000.0,retired,none
4,larry,25,37000.0,bodybuilder,none
5,patrick star,31,0.0,unemployed,none


#### Query Expression

You can define a boolean expression and pass it as a string to `DataFrame.query()` function to filter data

In [244]:
# Get all employed people (i.e., not unemployed and not retired)
data.query("profession != 'retired' and profession != 'unemployed'")

,name,age,salary_usd,profession,employer
0,spongebob,19,12000.0,chef,the krusty krab
1,plankton,45,2.0,entrepreneur,the chum bucket
3,mr. krabs,41,100000.0,entrepreneur,the krusty krab
4,larry,25,37000.0,bodybuilder,none


### Adding and Removing Data

#### Rows

In [245]:
# adding row
new_data = pd.DataFrame([{
    'name': 'squidward', 
    'age': 42, 
    'salary_usd': 10000,
    'profession': 'musician',
    'employer': 'the krusty krab'
}])
# convert the new data columns to correct types otherwise concat will revert our previous typing
# information
new_data['age'] = pd.to_numeric(data['age'], downcast="integer", errors="coerce")
new_data['salary_usd'] = pd.to_numeric(data['salary_usd'], downcast='float', errors='coerce')
# append the new data to the original dataset
data = pd.concat([data, new_data], ignore_index=True)
data



,name,age,salary_usd,profession,employer
0,spongebob,19,12000.0,chef,the krusty krab
1,plankton,45,2.0,entrepreneur,the chum bucket
2,sandy,24,65000.0,retired,none
3,mr. krabs,41,100000.0,entrepreneur,the krusty krab
4,larry,25,37000.0,bodybuilder,none
5,patrick star,31,0.0,unemployed,none
6,squidward,19,12000.0,musician,the krusty krab


In [246]:
# removing row
copy = data.copy()
dropped = copy.drop(index=1)
dropped

,name,age,salary_usd,profession,employer
0,spongebob,19,12000.0,chef,the krusty krab
2,sandy,24,65000.0,retired,none
3,mr. krabs,41,100000.0,entrepreneur,the krusty krab
4,larry,25,37000.0,bodybuilder,none
5,patrick star,31,0.0,unemployed,none
6,squidward,19,12000.0,musician,the krusty krab


#### Columns

In [247]:
# Adding Column
species = pd.Series(['sponge', 'plankton', 'squirrel', 'crab', 'lobster', 'starfish', 'octopus'], dtype=str)
data['species'] = species
data

,name,age,salary_usd,profession,employer,species
0,spongebob,19,12000.0,chef,the krusty krab,sponge
1,plankton,45,2.0,entrepreneur,the chum bucket,plankton
2,sandy,24,65000.0,retired,none,squirrel
3,mr. krabs,41,100000.0,entrepreneur,the krusty krab,crab
4,larry,25,37000.0,bodybuilder,none,lobster
5,patrick star,31,0.0,unemployed,none,starfish
6,squidward,19,12000.0,musician,the krusty krab,octopus


In [248]:
# Adding column using concat
home = pd.DataFrame({
    'home': [
        'pineapple',
        'bucket',
        'air enclosure',
        'anchor',
        'muscle beach',
        'under a rock',
        'moai statue'
    ]
})
data = pd.concat([data, home], axis=1)
data

,name,age,salary_usd,profession,employer,species,home
0,spongebob,19,12000.0,chef,the krusty krab,sponge,pineapple
1,plankton,45,2.0,entrepreneur,the chum bucket,plankton,bucket
2,sandy,24,65000.0,retired,none,squirrel,air enclosure
3,mr. krabs,41,100000.0,entrepreneur,the krusty krab,crab,anchor
4,larry,25,37000.0,bodybuilder,none,lobster,muscle beach
5,patrick star,31,0.0,unemployed,none,starfish,under a rock
6,squidward,19,12000.0,musician,the krusty krab,octopus,moai statue


In [249]:
# Removing Column
without_age = data.drop(columns=['age'])
without_age

,name,salary_usd,profession,employer,species,home
0,spongebob,12000.0,chef,the krusty krab,sponge,pineapple
1,plankton,2.0,entrepreneur,the chum bucket,plankton,bucket
2,sandy,65000.0,retired,none,squirrel,air enclosure
3,mr. krabs,100000.0,entrepreneur,the krusty krab,crab,anchor
4,larry,37000.0,bodybuilder,none,lobster,muscle beach
5,patrick star,0.0,unemployed,none,starfish,under a rock
6,squidward,12000.0,musician,the krusty krab,octopus,moai statue


#### Using `apply()`

In [250]:
# apply custom logic 
salary_bonus_func = lambda d: d + 123
copy.salary_usd[copy.employer == 'the krusty krab'].apply(salary_bonus_func)

0     12123.0
3    100123.0
6     12123.0
Name: salary_usd, dtype: float64

#### Handling Missing Data

Two strategies for handling missing data
1. Drop data
    * Drop Row
    * Drop Column
2. Impute/Substitute data
    * By Mean

##### Drop Data

In [251]:
with_missing_values = pd.DataFrame([
    [1,'a','hello'],
    [None, 'b', None],
    [3, 'c', None],
    [4, 'd', 'world']
], columns=['first', 'second', 'third'])
print(with_missing_values)

dropped_rows = with_missing_values.dropna(axis=0)
print(dropped_rows)

dropped_cols = with_missing_values.dropna(axis=1)
print(dropped_cols)

   first second  third
0    1.0      a  hello
1    NaN      b   None
2    3.0      c   None
3    4.0      d  world
   first second  third
0    1.0      a  hello
3    4.0      d  world
  second
0      a
1      b
2      c
3      d


You can customize other parameters of the drop operation such as the threshold (number of missing values) needed for a candidate row/column to be dropped

In [252]:
# drop rows with 2 or more missing values
with_missing_values.dropna(axis=0, thresh=2)    # keeps rows with >= 2 non-empty values

,first,second,third
0,1.0,a,hello
2,3.0,c,None
3,4.0,d,world


##### Impute/Substitute Data

In [253]:
first_col_mean = with_missing_values['first'].mean()
with_missing_values['first'].fillna(value=first_col_mean, inplace=True)
with_missing_values

,first,second,third
0,1.000000,a,hello
1,2.666667,b,None
2,3.000000,c,None
3,4.000000,d,world


In [254]:
with_missing_values['third'].fillna(value='IMPUTED', inplace=True)
with_missing_values

,first,second,third
0,1.000000,a,hello
1,2.666667,b,IMPUTED
2,3.000000,c,IMPUTED
3,4.000000,d,world


## Summarizing Data

In [255]:
print(len(data))                    # number of rows in dataframe
print(data['employer'].nunique())   # number of unique values in column
print(data['employer'].value_counts())  # tally of the unique values
data.describe()


7
3
the krusty krab     3
none                3
 the chum bucket    1
Name: employer, dtype: int64


,age,salary_usd
count,7.000000,7.000000
mean,29.142857,32286.000000
std,10.367072,37827.017311
min,19.000000,0.000000
25%,21.500000,6001.000000
50%,25.000000,12000.000000
75%,36.000000,51000.000000
max,45.000000,100000.000000


Additional summary functions are available. They operate on different types (DataFrame, Series, etc) and return a single value for each group.

When applied to a DataFrame, these functions return a Series containing the 


In [256]:
copy = data.copy(deep=True)
ages = copy.age

print(ages.mean())      # column mean
print(ages.median())    # column median
print(ages.min())       # minimum
print(ages.max())       # maximum
print(ages.sum())       # column sum
print(ages.var())       # variance
print(ages.std())       # standard deviation
print(ages.quantile([0.25, 0.75]))  


29.142857142857142
25.0
19
45
204
107.47619047619048
10.367072415884365
0.25    21.5
0.75    36.0
Name: age, dtype: float64


## Grouping Data and Aggregate Functions

Aggregate Functions: In this context, functions that take a series of values and returns a summary. Examples of aggregate functions are:
* `sum()`
* `mean()`

They are commonly used in conjunction with grouped data represented by `DataFrameGroupBy` or `SeriesGroupBy` instances.

Helpful Resources:
* https://pbpython.com/groupby-agg.html

In [257]:
# Apply aggregate function on dataframe
data.salary_usd.agg([np.min, np.max, np.mean, np.std])

amin         0.000000
amax    100000.000000
mean     32286.000000
std      37827.017311
Name: salary_usd, dtype: float64

In [258]:
# Group records based on employer then apply aggregate function
kk_employees = data.groupby(by='employer')      # by default this will include all numeric columns

kk_employees.agg({
    'age': 'mean',
    'salary_usd': ['mean', 'count']     # 'count' will not include NaN  values, 'size' will
})

age    salary_usd      
                       mean          mean count
employer                                       
 the chum bucket  45.000000      2.000000     1
none              26.666667  34000.000000     3
the krusty krab   26.333333  41333.333333     3

## Joining Data

In [259]:
# Prepare 2 dataframes for join operations
data_a = pd.DataFrame({
    'col': ['1','2','3'],
    'cola': ['aa', 'bb', 'cc']
}, index=['a', 'b' , 'c'])
print(data_a)

data_b = pd.DataFrame({
    'col': ['1','3','4'],
    'colb': [True, False, True]
}, index=['a', 'c', 'd'])
data_b
print(data_b)


  col cola
a   1   aa
b   2   bb
c   3   cc
  col   colb
a   1   True
c   3  False
d   4   True


### Inner Join

In [260]:
# Inner Join (based on index values)
pd.merge(data_a, data_b, left_index=True, right_index=True)

,col_x,cola,col_y,colb
a,1,aa,1,True
c,3,cc,3,False


In [261]:
# Inner Join (based on column value)
pd.merge(data_a, data_b, how='inner', on='col')

,col,cola,colb
0,1,aa,True
1,3,cc,False


### Left Join

In [262]:
# Left Join (based on index)
pd.merge(data_a, data_b, how='left', left_index=True, right_index=True)

,col_x,cola,col_y,colb
a,1,aa,1,True
b,2,bb,NaN,NaN
c,3,cc,3,False


In [263]:
# Left Join (based on column)
pd.merge(data_a, data_b, how='left', on='col')

,col,cola,colb
0,1,aa,True
1,2,bb,NaN
2,3,cc,False


### Right and Outer Join

Right and Outer Join examples are omitted as they function the same as the `Left Join` examples above except with `how='right'` and `how='outer'` used instead, respectively
